In [4]:
import pandas as pd
import numpy as np
import re
import os
from fuzzywuzzy import fuzz,process

In [5]:
pd.set_option('display.max_rows', 2000)
path = 'C:/Users/HP/Documents/DQ & Regex demo/'

In [6]:
client = pd.read_excel(os.path.join(path,'Inputs/client_data_week1.xlsx'))
employee_table = pd.read_excel(os.path.join(path,'Inputs/Employee Table.xlsx'))

In [7]:
employee_table

,SAP_ID,Employee names
0,16001,"LEWIS, GABE"
1,16002,"SCOTT, MICHAEL GARY"
2,16003,"HALPERT, JAMES JIM"
3,16004,"BEESLY, PAMELA MORGAN"
4,16005,"SCHRUTE, DWIGHT KURT"
5,16006,"BERNARD, ANDY BAINES"
6,16007,"CALIFORNIA, ROBERT"
7,16010,"MALONE1, KEVIN"
8,16011,"MALONE2, KEVIN"
9,16015,"FLENDERSON, TOBY"


In [8]:
client

,CLIENT_ID,CLIENT_NAME,CLIENT_LOCATION,CLIENT_INDUSTRY,CLIENT_MANAGER_ID,CLIENT_MANAGER_NAME
0,220101,DNNSV Group,UNITED KINGDOM,INTERNAL,16006,Andy Baines Bernard
1,220102,Sportswear,UK,RETAIL,16010,"Malone 1,Kevin"
2,220103,AGNIA UK,UNITED KINGDOM,BANKING,16004,"pamela morgan beesly; BEESLY, PAMELA MORGAN"
3,220104,GGB Group,SPAIN,BANKING,16010,"Malone 1,Kevin"
4,220105,NikitasAE,Austria,BANKING,16006,Andy Baines Bernard
5,220106,Nomitra,Belgium,CAPITAL MARKETS,16003,"Halpert, James Jim (CO-MANAGER)"
6,220107,ECWFMU,France,BANKING,16004,Pamela Morgan Beesly
7,220108,PSS Gent,Germany,PROPERTY,16006,Andy Baines Bernard
8,220109,Atotis,Liechtenstein,INTERNAL,16038,Danny Cordray
9,220110,WorkHardLTD,Luxembourg,BANKING,16003,James Jim Halpert


### Data Quality Check: We want to ensure that the employee names and the corresponding employee id have a 1:1 relationship. We want to see if the employee id is really a unique identifier. 

#### Are many people sharing 1 employee id?
#### Are people assigned the wrong employee id?


In [11]:
# DQ Check 1:1 Mapping between employee id and manager name
client['ID_IS_UNIQUE'] = client.groupby(['CLIENT_MANAGER_ID'])['CLIENT_MANAGER_NAME'].transform(lambda x: x.nunique() == 1)
id_check = client.loc[client['ID_IS_UNIQUE'] == False, ['CLIENT_MANAGER_ID','CLIENT_MANAGER_NAME','ID_IS_UNIQUE']].sort_values(['CLIENT_MANAGER_ID'])

In [12]:
id_check.merge(employee_table, how = 'left', left_on = 'CLIENT_MANAGER_ID',right_on = 'SAP_ID')

,CLIENT_MANAGER_ID,CLIENT_MANAGER_NAME,ID_IS_UNIQUE,SAP_ID,Employee names
0,16001,Gabe Lewis,False,16001,"LEWIS, GABE"
1,16001,Gabe Lewis,False,16001,"LEWIS, GABE"
2,16001,Gabe Lewis,False,16001,"LEWIS, GABE"
3,16001,Gabe Lewis AND James Jim Halpert,False,16001,"LEWIS, GABE"
4,16001,Gabe Lewis,False,16001,"LEWIS, GABE"
5,16003,"Halpert, James Jim (CO-MANAGER)",False,16003,"HALPERT, JAMES JIM"
6,16003,James Jim Halpert,False,16003,"HALPERT, JAMES JIM"
7,16003,James Jim Halpert,False,16003,"HALPERT, JAMES JIM"
8,16003,"Halpert, James Jim (CO-MANAGER)",False,16003,"HALPERT, JAMES JIM"
9,16003,James Jim Halpert,False,16003,"HALPERT, JAMES JIM"


##### - Notice on rows 65 to 74, we find that Michael Scott has been assigned an employee id that is not his. It turns out that Ben Nugent and A.J. have left the company and Micheal has taken over their clientele. 

##### - However due to bad file management nobody changed the employee id. 

#### Problem: Management have asked us to produce a dashboard showing the employee and the progress status of their engagements.

##### -  We could manually reassign these but if we had hundreds of cases like this, we would need to override the employee id based on the employee table. 
##### -  The problem is that we need a unique identifier to do this. However if the unique id is corrupted then we can only rely on joining based on name column. 

##### -  This is a horrible idea since employee names are not in the same format so joining the employee table based on employee name would never work.

### Q) So how can we produce our dashboard showing employees vs their engagement progress?
### Solution: We can massage the data using REGULAR EXPRESSION to standardise the naming formats improving dashboard quality which would make our stakeholders very happy as it reduces their work.


In [13]:
def name_cleaner(df,col,identifier):
    #ideal format: LASTNAME, FIRSTNAME or LASTNAME, FIRSTNAME MIDDLENAMES
    
    #case1: FIRSTNAME LASTNAME
    rows = df.shape[0]
    
    df['CLEANED_'+ str(col)] = df[col].str.upper()
    
    case1 = df.loc[df[col].str.contains(';|\s+AND+\s|,|/|\.|\?|@', regex = True) == False, [identifier, 'CLEANED_'+str(col)]]
    
    if case1.shape[0] != 0:
        case1['CASE1_CLEANED'] = np.where(case1['CLEANED_'+str(col)].str.count(' ') == 1,
                                         case1['CLEANED_'+str(col)].str.replace('^.+\s','', regex = True) + ', ' + case1['CLEANED_'+str(col)].str.replace('\s+[A-z]{2,}','',regex = True),
                                         case1['CLEANED_'+str(col)].str.replace('^.+\s','', regex = True) + ', ' + case1['CLEANED_'+str(col)].str.replace('\s+[A-z]+','', regex = True) + ' ' + case1['CLEANED_'+str(col)].replace('^\S+\s+|\s+\S+$','', regex = True))
        df = pd.merge(df,case1[[identifier,'CASE1_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE1_CLEANED'].notnull() == True, df['CASE1_CLEANED'], df['CLEANED_'+str(col)])
                                         
    else:
        print('There are no FIRSTNAME LASTNAME patterns.')
        
    assert(df.shape[0] == rows, 'Duplication occured in case 1. Check')
    
    # CASE 2: lastname,Firstname (no space)
                                         
    case2 = df.loc[(df['CLEANED_'+str(col)].str.contains('\S+,+\S+', regex= True) == True) & (df['CLEANED_'+str(col)].str.contains(';') == False), [identifier, 'CLEANED_'+str(col)]]
    
    if case2.shape[0] != 0:
        case2['CASE2_CLEANED'] = case2['CLEANED_'+str(col)].str.replace(',',', ', regex = True)
        df = pd.merge(df,case2[[identifier,'CASE2_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE2_CLEANED'].notnull() == True, df['CASE2_CLEANED'], df['CLEANED_'+str(col)])
    else:
        print('There are no Lastname,Firstname without space.')
        
    assert(df.shape[0] == rows, 'Duplication occured in case 2. Check')

    # CASE 3: contains AND or /
    case3 = df.loc[df['CLEANED_'+str(col)].str.contains('\s+AND+\s|\s*/+\s*|\S+/', regex = True) == True, [identifier, 'CLEANED_'+ str(col)]]
    
    if case3.shape[0] != 0:
        case3['CASE3_CLEANED'] = case3['CLEANED_'+str(col)].str.replace('\s+AND+\s|\s*/+\s*',';', regex = True)
        #case3['CASE3_CLEANED'] = case3['CASE3_CLEANED'].str.replace('; ',';')
        df = pd.merge(df,case3[[identifier,'CASE3_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE3_CLEANED'].notnull() == True, df['CASE3_CLEANED'], df['CLEANED_'+str(col)])
    else:
        print('There are no AND or / cases.')
        
    assert(df.shape[0] == rows, 'Duplication occured in case 3. Check')
                    
    # CASE 4: brackets
    case4 = df.loc[df['CLEANED_'+str(col)].str.contains('\(\?|[A-z]+\)',regex= True) == True, [identifier, 'CLEANED_'+str(col)]]
    
    if case4.shape[0] != 0:
        case4['CASE4_CLEANED'] = case4['CLEANED_'+str(col)].str.replace('\(\?+\)','', regex = True)
        case4['CASE4_CLEANED'] = case4['CLEANED_'+str(col)].str.replace('\s*\(+[A-z]+\)','', regex = True)
        case4['CASE4_CLEANED'] = case4['CLEANED_'+str(col)].str.replace('\s*\(+[A-z]+.+\)','', regex = True)
                
        df = pd.merge(df,case4[[identifier,'CASE4_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE4_CLEANED'].notnull() == True, df['CASE4_CLEANED'], df['CLEANED_'+str(col)])
    else:
        print('There are no bracket cases.')
    
    assert(df.shape[0] == rows, 'Duplication occured in case 4. Check')
    
    #  CASE 5: Numbers
    case5 = df.loc[(df['CLEANED_'+str(col)].str.contains('[0-9]', regex = True)) & (df['CLEANED_'+ str(col)].str.contains('\(', regex = True) == False), [identifier, 'CLEANED_'+str(col)]]
    
    if case5.shape[0] != 0:
        case5['CASE5_CLEANED'] = np.where(case5['CLEANED_'+str(col)].str.contains('[A-z]\s+[0-9]', regex = True),
        case5['CLEANED_'+str(col)].str.replace('(?<=\D)\s+(?=\d)','', regex = True),
        case5['CLEANED_'+str(col)])
                    
        df = pd.merge(df,case5[[identifier,'CASE5_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE5_CLEANED'].notnull() == True, df['CASE5_CLEANED'], df['CLEANED_'+str(col)])
    else:
        print('There are no numbers in text.')
    
    assert(df.shape[0] == rows, 'Duplication occured in case 5. Check')
    
    # CASE 6: email addresses
    case6 = df.loc[df['CLEANED_'+str(col)].str.contains('<.+@.+>', regex = True), [identifier, 'CLEANED_'+str(col)]]
    
    if case6.shape[0] != 0:
        case6['CASE6_CLEANED'] = case6['CLEANED_'+str(col)].str.replace('<.+@.+>','', regex = True)
        df = pd.merge(df,case6[[identifier,'CASE6_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE6_CLEANED'].notnull() == True, df['CASE6_CLEANED'], df['CLEANED_'+str(col)])
                    
    else:
        print('There are no email patterns.')
    
    assert(df.shape[0] == rows, 'Duplication occured in case 6. Check')
    
     # CASE 7: ; clean up
                    
    df['CLEANED_'+str(col)] = df['CLEANED_'+str(col)].str.replace('; ',';')
    case7 = df.loc[df['CLEANED_'+str(col)].str.contains('.*;.*', regex = True) == True, [identifier, 'CLEANED_'+str(col)]]
    if case7.shape[0] != 0:
        case7 = pd.concat([case7[identifier],case7['CLEANED_'+str(col)], case7['CLEANED_'+ str(col)].str.split(';',expand = True)],axis = 1)
        case7.reset_index(drop = True, inplace = True)
        
        def spacer(x):
            if x is not None:
                    if bool(re.search(' ',str(x)) == False):
                        return re.sub(',',', ',x)
                    elif bool(re.search('\S+,+\S+',str(x))) == True:
                        return re.sub(',',', ',x)
                    else:
                        return x
            else:
                return x
        columns = list(set(list(case7.columns)) - set([identifier,'CLEANED_'+str(col)]))
        
        for i in columns:
            case7[i] = case7[i].apply(spacer)
        
        # remove duplicates across rows
        dups = case7.apply(pd.Series.duplicated,axis = 1) #marks duplicates across rows true except the first
        case7 = case7.mask(dups, None)  #replaces the true values with None
        
        # merging the different columns together except nones
        case7['CASE7_CLEANED'] = case7[columns].apply(lambda x: ';'.join(x.dropna()), axis = 1)
        
        # joining cleaned column back to the dataset
        df = pd.merge(df, case7[[identifier,'CASE7_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+str(col)] = np.where(df['CASE7_CLEANED'].notnull() == True,df['CASE7_CLEANED'],df['CLEANED_'+str(col)])
        
        assert(df.shape[0] == rows, 'Duplication occured in case 7. Check')
        #CASE 8: FIRSTNAME LASTNAME;LASTNAME, FIRSTNAME  --> LASTNAME, FIRSTNAME; LASTNAME, FIRSTNAME
                    
        case8 = case7.loc[(case7['CASE7_CLEANED'].str.contains('^[A-z]+(?!,)(?=\s)\s') == True)
                          & (case7['CASE7_CLEANED'].str.contains(';',regex = True) == True),[identifier, 'CLEANED_'+str(col), 'CASE7_CLEANED']]
        case8.reset_index(drop = True, inplace = True)
        case8 = pd.concat([case8[identifier],case8['CASE7_CLEANED'],case8['CASE7_CLEANED'].str.split(';',expand = True)], axis = 1)
        columns_case8 = list(set(list(case8.columns)) - set([identifier,'CLEANED_'+str(col),'CASE7_CLEANED']))
        case8[columns_case8] = case8[columns_case8].astype(str)
        
        for i in columns_case8:
            case8[i] = case8[i].str.lstrip()
            case8[i] = case8[i].str.rstrip()
        
        def standardiser(x):
            if x != None:
                if bool(re.search(',',str(x))) == False:
                    if len(re.findall(' ',str(x))) == 1:
                        return re.sub('^.+\s','',x) + ', ' + re.sub('\s+[A-z]{2,}','',x)
                    else:
                        return re.sub('^.+\s','',x) + ', ' + re.sub('\s+[A-z]{2,}','',x) + ' ' + re.sub('^\S+\s+|\s+\S+$','',x)
                else:
                    return x
            else:
                return x
                    
        for i in columns_case8:
            case8[i] = case8[i].apply(standardiser)
         # remove any duplicates: assign Nones to duplicates
        dups2 = case8.apply(pd.Series.duplicated, axis = 1)
        case8 = case8.mask(dups2, None)
        
        #remove any duplicates: (turn string nones into none)
        for i in columns_case8:
                    for j in range(case8.shape[0]):
                        if case8.loc[case8.index[j],i] == 'None':
                            case8.loc[case8.index[j],i] = None
        
        # remove any duplicates: Merge all columns except None:
        case8['CASE8_CLEANED'] = case8[columns_case8].apply(lambda x: ';'.join(x.dropna()), axis = 1)
        
        # joining to main dataset
        df = pd.merge(df,case8[[identifier,'CASE8_CLEANED']], how = 'left', on = identifier)
        df['CLEANED_'+ str(col)] = np.where(df['CASE8_CLEANED'].notnull() == True, df['CASE8_CLEANED'],df['CLEANED_'+ str(col)])
        
        assert(df.shape[0] == rows, 'Duplication occured in case 7. Check')
    else:
        print('There are no ; cases.')
    
        
    return df

In [14]:
client_clean = name_cleaner(client, 'CLIENT_MANAGER_NAME','CLIENT_ID')

### Checking the unique employee names and if they satisfy are naming conventions.

In [15]:
client_clean['CLEANED_CLIENT_MANAGER_NAME'].drop_duplicates().to_frame().reset_index(drop = True)

,CLEANED_CLIENT_MANAGER_NAME
0,"BERNARD, ANDY BAINES"
1,"MALONE1, KEVIN"
2,"BEESLY, PAMELA MORGAN"
3,"HALPERT, JAMES JIM"
4,"CORDRAY, DANNY"
5,"SCHRUTE, DWIGHT KURT"
6,"LEWIS, GABE;HALPERT, JAMES JIM"
7,"SCOTT, MICHAEL"
8,"CALIFORNIA, ROBERT"
9,"LEWIS, GABE"


#### Notice that there are 2 names that we were unable to standardise through regular expression. These are SCOTT, MICHAEL GARY ad DWIGHTKURT, SCHRUTE. 

#### In one instance, an employee's middle name was included and in the other an employee's name was misspelt. 

#### Regular expression can only search and locate patterns, it cannot discover the similarities between names and override them. At a generalised code level, SCOTT, MICHAEL GARY and SCOTT, MICHEAL are two different people but we know that they are same.

### Q) How can we fix this issue?

### Solution: Fuzzy string matching via FuzzyWuzzy library. 

#### Fuzzy String Matching, also known as Approximate String Matching, is the process of finding strings that approximately match a pattern. The process has various applications such as spell-checking, DNA analysis and detection, spam detection, plagiarism detection e.t.c

#### The Fuzzywuzzy library uses tein Distance to calculate the differences between sequences and patterns that was developed

In [16]:
unique_check2 = client_clean[['CLIENT_MANAGER_ID','CLEANED_CLIENT_MANAGER_NAME']].drop_duplicates().sort_values(['CLIENT_MANAGER_ID'])
unique_check2['IS_UNIQUE'] = unique_check2.groupby(['CLIENT_MANAGER_ID'])['CLEANED_CLIENT_MANAGER_NAME'].transform(lambda x: x.nunique() == 1)
unique_check2.loc[unique_check2['IS_UNIQUE'] == False,]

,CLIENT_MANAGER_ID,CLEANED_CLIENT_MANAGER_NAME,IS_UNIQUE
17,16001,"LEWIS, GABE;HALPERT, JAMES JIM",False
36,16001,"LEWIS, GABE",False
14,16005,"SCHRUTE, DWIGHT KURT",False
43,16005,"SCHRUTE, DWIGHT",False
60,16039,"SCOTT, MICHAEL",False
63,16039,"SCOTT, MICHAEL GARY",False
31,16040,"SCOTT, MICHAEL",False
71,16040,"SCOTT, MICHAEL GARY",False


### The fuzz.ratio method simply scores based on the edit distance.
### The fuzz.partial_ratio matches with sub strings and then gives a score
### The token sort ratio method attempts to account for similar strings that are out of order. 
### In our case the best metric would be token sort ratio as we have strings out of order and some missing string components.

In [17]:
def renamer(data,name,accuracy): 
        x = fuzz.token_sort_ratio(name,data)
        if x >= accuracy:
            output = name
        else:
            output = data
        return output
        

In [18]:
client_clean['CLEANED_CLIENT_MANAGER_NAME'] = client_clean['CLEANED_CLIENT_MANAGER_NAME'].apply(renamer,name = 'SCOTT, MICHAEL GARY',accuracy = 80)
client_clean['CLEANED_CLIENT_MANAGER_NAME'] = client_clean['CLEANED_CLIENT_MANAGER_NAME'].apply(renamer,name = 'SCHRUTE, DWIGHT KURT',accuracy = 80)
client_clean['CLEANED_CLIENT_MANAGER_NAME'].drop_duplicates().to_frame().reset_index(drop = True)

,CLEANED_CLIENT_MANAGER_NAME
0,"BERNARD, ANDY BAINES"
1,"MALONE1, KEVIN"
2,"BEESLY, PAMELA MORGAN"
3,"HALPERT, JAMES JIM"
4,"CORDRAY, DANNY"
5,"SCHRUTE, DWIGHT KURT"
6,"LEWIS, GABE;HALPERT, JAMES JIM"
7,"SCOTT, MICHAEL GARY"
8,"CALIFORNIA, ROBERT"
9,"LEWIS, GABE"


In [ ]:
## Check equality of expected number of distinct names vs actual number of distinct names

In [19]:
client_clean['CLEANED_CLIENT_MANAGER_NAME'].drop_duplicates().to_frame().reset_index(drop = True).shape[0] == employee_table.shape[0] -2 + 1

True

In [1]:
## Lets look at which names are still not unique based on employee id

In [20]:
unique_check3 = client_clean
unique_check3['IS_UNIQUE'] = unique_check3.groupby(['CLIENT_MANAGER_ID'])['CLEANED_CLIENT_MANAGER_NAME'].transform(lambda x: x.nunique() == 1)
unique_check3.loc[unique_check3['IS_UNIQUE'] == False, ['CLIENT_MANAGER_ID','CLEANED_CLIENT_MANAGER_NAME']]

,CLIENT_MANAGER_ID,CLEANED_CLIENT_MANAGER_NAME
17,16001,"LEWIS, GABE;HALPERT, JAMES JIM"
36,16001,"LEWIS, GABE"
79,16001,"LEWIS, GABE"
95,16001,"LEWIS, GABE"
101,16001,"LEWIS, GABE"


In [ ]:
# Export data to inspect if needed

In [21]:
client_clean.to_excel(os.path.join(path,'week1_cleaned.xlsx'),index = False)